 Licensed Materials - Property of IBM (c) Copyright IBM Corp. 2024 All Rights Reserved.

 US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with
 IBM Corp.

 DISCLAIMER OF WARRANTIES :

 Permission is granted to copy and modify this Sample code, and to distribute modified versions provided that both the
 copyright notice, and this permission notice and warranty disclaimer appear in all copies and modified versions.

 THIS SAMPLE CODE IS LICENSED TO YOU AS-IS. IBM AND ITS SUPPLIERS AND LICENSORS DISCLAIM ALL WARRANTIES, EITHER
 EXPRESS OR IMPLIED, IN SUCH SAMPLE CODE, INCLUDING THE WARRANTY OF NON-INFRINGEMENT AND THE IMPLIED WARRANTIES OF
 MERCHANTABILITY OR FITNESS FOR A PARTICULAR PURPOSE. IN NO EVENT WILL IBM OR ITS LICENSORS OR SUPPLIERS BE LIABLE FOR
 ANY DAMAGES ARISING OUT OF THE USE OF OR INABILITY TO USE THE SAMPLE CODE, DISTRIBUTION OF THE SAMPLE CODE, OR
 COMBINATION OF THE SAMPLE CODE WITH ANY OTHER CODE. IN NO EVENT SHALL IBM OR ITS LICENSORS AND SUPPLIERS BE LIABLE
 FOR ANY LOST REVENUE, LOST PROFITS OR DATA, OR FOR DIRECT, INDIRECT, SPECIAL, CONSEQUENTIAL, INCIDENTAL OR PUNITIVE
 DAMAGES, HOWEVER CAUSED AND REGARDLESS OF THE THEORY OF LIABILITY, EVEN IF IBM OR ITS LICENSORS OR SUPPLIERS HAVE
 BEEN ADVISED OF THE POSSIBILITY OF SUCH DAMAGES.

In [ ]:
import json
import config
from demoutil import ClassDefinitionsSelector
from csdeploy import Importer, Exporter, GraphqlConnection, DeploymentPackage, ExportSelection, logging

# Source system config

In [ ]:
print(config.GQL_URL)
print(config.OS_NAME)

# Set Logging level (Optional)

Note: Change level to `logging.DEBUG` for more log details on requests being made

In [ ]:
logging.basicConfig()
logging.getLogger("csdeploy").setLevel(logging.WARNING)

# Authenticate with the source GraphQL server
Choose one method of Authentication <br> <br>
Note: Set `ssl_enabled = false` in `GraphqlConnection` if SSL issue occurs

#### OAuth Authentication

In [ ]:
source_connection = GraphqlConnection(url=config.GQL_URL, token_url=config.TOKEN_URL)
source_connection.initialize_oauth(client_id=config.CLIENT_ID,client_secret= config.CLIENT_SECRET, username=config.OAUTH_USERNAME, password=config.OAUTH_PASSWORD,\
                                grant_type=config.GRANT_TYPE, scope=config.SCOPE)
source_connection.get_token()

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("XSRF token: " + source_connection.xsrf_token)
print("Authentication token: " + source_connection.token)

#### Basic Authentication

In [ ]:
source_connection = GraphqlConnection(url=config.GQL_URL)
source_connection.initialize_basic(username=config.BASIC_USERNAME, password=config.BASIC_PASSWORD)

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("XSRF token: " + source_connection.xsrf_token)

#### Zen and IAM Authentication

In [ ]:
source_connection = GraphqlConnection(url=config.GQL_URL, token_url=config.TOKEN_URL)
source_connection.initialize_zen_iam(zen_exchange_url=config.EXCHANGE_TOKEN_URL, iam_client_id=config.CLIENT_ID,\
                                    iam_client_secret= config.CLIENT_SECRET, iam_username=config.OAUTH_USERNAME,\
                                    iam_password=config.OAUTH_PASSWORD, iam_grant_type=config.GRANT_TYPE,\
                                    iam_scope=config.SCOPE)
source_connection.get_token()

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("XSRF token: " + source_connection.xsrf_token)
print("Access token: " + source_connection.token)

# Prompt for one or more classes to export

In [ ]:
selector = ClassDefinitionsSelector(config.OS_NAME, source_connection)
selector.prompt_selection()
print("Classes selected:")
for sel in selector.selections:
    jsonobj = sel.to_json()
    print(str(jsonobj))
export_selection = ExportSelection(class_definitions=selector.selections)

# Exporter

#### Initialize an Exporter object with the classes to export

In [ ]:
deployment_package = DeploymentPackage()
exporter = Exporter(source_connection, config.OS_NAME, deployment_package)
exporter.initialize_selection(export_selection)

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
pkg_json = deployment_package.to_json()
print("Deployment package after selection:")
print(json.dumps(pkg_json, indent=4))

#### Export objects in the package

In [ ]:
exporter.export()

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("Deployment package after objects exported:")
print(json.dumps(deployment_package.to_json(), indent=4))

#### Save the package to a file

In [ ]:
deployment_package.write("export.json")

# Target System Config

In [ ]:
print(config.TARGET_GQL_URL)
print(config.TARGET_OS_NAME)

# Authenticate with the target GraphQL server
(Choose one method of Authentication) <br><br>
Note: Set `ssl_enabled = false` in `GraphqlConnection` if SSL issue occurs

#### OAuth Authentication

In [ ]:
target_connection = GraphqlConnection(url=config.TARGET_GQL_URL, token_url=config.TARGET_TOKEN_URL)
target_connection.initialize_oauth(client_id=config.TARGET_CLIENT_ID,client_secret= config.TARGET_CLIENT_SECRET, username=config.TARGET_OAUTH_USERNAME, password=config.TARGET_OAUTH_PASSWORD,\
                                grant_type=config.TARGET_GRANT_TYPE, scope=config.TARGET_SCOPE)
target_connection.get_token()

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("XSRF token: " + target_connection.xsrf_token)
print("Authentication token: " + target_connection.token)

#### Basic Authentication

In [ ]:
target_connection = GraphqlConnection(url=config.TARGET_GQL_URL, ssl_enabled=False)
target_connection.initialize_basic(username=config.TARGET_BASIC_USERNAME, password=config.TARGET_BASIC_PASSWORD)

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("XSRF token: " + target_connection.xsrf_token)

#### Zen and IAM Authentication

In [ ]:
target_connection = GraphqlConnection(url=config.TARGET_GQL_URL, token_url=config.TARGET_TOKEN_URL)
target_connection.initialize_zen_iam(zen_exchange_url=config.TARGET_EXCHANGE_TOKEN_URL, iam_client_id=config.TARGET_CLIENT_ID,\
                                    iam_client_secret= config.TARGET_CLIENT_SECRET, iam_username=config.TARGET_OAUTH_USERNAME,\
                                    iam_password=config.TARGET_OAUTH_PASSWORD, iam_grant_type=config.TARGET_GRANT_TYPE,\
                                    iam_scope=config.TARGET_SCOPE)
target_connection.get_token()

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("XSRF token: " + target_connection.xsrf_token)
print("Access token: " + target_connection.token)

# Importer

#### Load the package file previously exported

In [ ]:
deployment_package = DeploymentPackage()
deployment_package.read('export.json')

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print(json.dumps(deployment_package.to_json(), indent=4))

#### Generate and execute queries and mutations.
Note: To generate mutations before executing, call `Importer.generate_mutation()` method


In [ ]:
importer = Importer(target_connection, config.TARGET_OS_NAME, deployment_package)
mutrespjsons = importer.import_package()

#CODE BELOW IS ONLY USED FOR PRINT STATEMENTS
print("\nGenerated import discovery queries:")
for impqry in importer.discovery_queries:
    print("--------------------")
    print("Objects retrieved by query:")
    for rid in impqry.retrievals:
        print("    " + rid)
    print("The query:")
    print(impqry.query)
print("\nGenerated import mutations:")
for impmut in importer.import_mutations:
    print("--------------------")
    print("Objects modified by mutation:")
    for rid in impmut.modifications:
        print("    " + rid)
    print("Description of mutation: " + impmut.description)
    print("The mutation:")
    print(impmut.mutation)
print("Mutation responses:")

for mutrespjson in mutrespjsons:
    print("--------------------")
    print(json.dumps(mutrespjson, indent=4))